## Read and clean the treatment episode data

In [1]:
# Import related libraries
import pandas as pd
import uuid

In [2]:
# Load the CSV file into a DataFrame
episode = pd.read_csv('file path')
episode.shape

(1222644, 6)

We have 1.2M unique episodes in this data file with GAP = 30 days.

In [3]:
# Create unique ID for each patient's episodes
episode['uuid'] = [uuid.uuid4() for _ in range(len(episode.index))]

# Ensure the 'episode.start' column to datetime
episode['episode.start'] = pd.to_datetime(episode['episode.start'])

# Ensure the 'episode.end' column is a datetime
episode['episode.end'] = pd.to_datetime(episode['episode.end'])

# Min 'episode.start' is greater than 2014
# filtered_episode = episode[episode['episode.start'].dt.year >= 2014]
print("Earliest Date:", episode['episode.start'].min())

Earliest Date: 2014-01-01 00:00:00


## Read and clean the buprenorphine prescription data

In [4]:
# Load the buprenorphine prescription file
# Already did some cleaning and this is the data used for building the treatment episodes
RX = pd.read_csv("file path")
RX.shape

(16276713, 10)

We have a total of 16,276,713 RXs.

In [5]:
# Double check unique value for strength
print(set(RX['STRNT_DESC']))

{'8-2MG', '12-3 MG', '4-1 MG', '8-2 MG', '2-0.5 MG'}


#### We will clean up the RXs by keeping records with positive value for days supply and dispensed quantity. Then we will calculate the average daily dosage.

In [6]:
# Double check
# Filter positive value for days supply and quantity dispensed
RX = RX[(RX['DAYS_SUPPLY_CNT'] >0) & (RX['DSPNSD_QTY'] > 0)]
RX.shape

(16276713, 10)

In [7]:
# Extract the first value from the STRNT_DESC column 
# If the format is "a-b UNIT", extract 'a'. If the format is "a UNIT", keep 'a'
RX['STRNT_Bup'] = RX['STRNT_DESC'].apply(lambda x: x.split('-')[0] if '-' in x else x.split()[0])

# Ensure that the STRNT_Bup column is numeric
RX['STRNT_Bup'] = pd.to_numeric(RX['STRNT_Bup'], errors='coerce')

print(set(RX['STRNT_Bup']))

{8, 2, 12, 4}


In [8]:
# Calculate the dosage as STRNT_Bup * DSPNSD_QTY / DAYS_SUPPLY_CNT
RX['dosage'] = RX['STRNT_Bup'] * (RX['DSPNSD_QTY'] / RX['DAYS_SUPPLY_CNT'])

# How many % of the records have dosage>40
print(len(RX[RX['dosage']>40])/len(RX))

0.0011995665218155532


In [9]:
# Check number of duplicates
print(RX.duplicated().sum())
# Remove duplicates if needed
# RX = RX.drop_duplicates(keep=False)
len(RX)

0


16276713

We have 16,276,713 filtered RXs. 

## Merge treatment episode data and buprenorphine RXs data

In [10]:
# Merging the episode data and RXs data
episode_RX = episode.merge(RX, on='PATIENT_ID', how='inner')

In [11]:
# First, make sure the SVC_DT is datetime
episode_RX['SVC_DT'] = pd.to_datetime(episode_RX['SVC_DT'].astype(str), format='%Y%m%d').dt.normalize()

# Second, only keep the matched records when the RXs fall within an episode start date and episode end date
episode_RX = episode_RX[(episode_RX['SVC_DT'] >= episode_RX['episode.start']) & (episode_RX['SVC_DT'] <= episode_RX['episode.end'])]
len(episode_RX)

16276713

All 16,276,713 RXs successfully merged back with episode data.

In [12]:
# Make sure these variables are in the datetime format
episode_RX['episode.start'] = pd.to_datetime(episode_RX['episode.start'])
episode_RX['episode.end'] = pd.to_datetime(episode_RX['episode.end'])

In [13]:
# Check how many episodes involves a dosage>40
epi_dosage40 = episode_RX[episode_RX['dosage']>40]

# Double check to see whether it matches with the previous check at RX level
print(len(epi_dosage40)/len(episode_RX))
print(len(set(epi_dosage40['uuid'])))

0.0011995665218155532
8832


In [14]:
# Remove these records from the future analyses
episode_RX_keep = episode_RX[~episode_RX.uuid.isin(epi_dosage40.uuid)]
print(len(episode_RX_keep))
print(len(set(episode_RX_keep['uuid'])))

15606863
1213812


We initially have 1,222,644 episodes, 8,832 involve a dosage>40; after excluding these we have 1,213,812 episodes left; the data file with episodes and RXs merged now have 15,606,863 recrods.

In [15]:
# Double check
episode_RX_keep[["STRNT_Bup", "DSPNSD_QTY","dosage"]].describe()

,STRNT_Bup,DSPNSD_QTY,dosage
count,1.560686e+07,1.560686e+07,1.560686e+07
mean,7.665011e+00,2.898492e+01,1.427824e+01
std,1.662581e+00,2.457462e+01,5.940885e+00
min,2.000000e+00,5.000000e-01,5.714286e-02
25%,8.000000e+00,1.100000e+01,8.000000e+00
50%,8.000000e+00,2.100000e+01,1.600000e+01
75%,8.000000e+00,4.500000e+01,1.600000e+01
max,1.200000e+01,1.680000e+02,4.000000e+01


## Create max dosage value based on different time points

In [17]:
# Group by 'uuid'
groupedby_PI_ES = episode_RX_keep.groupby(by=['uuid'])

In [18]:
required_months = [1, 3, 6, 12, 18]

# Define a function to calculate the maximum dosage for each period
def calculate_max_dosage(group, periods):
    group = group.sort_values('SVC_DT')
    uuid = group['uuid'].iloc[0]  # Get the group identifier
    results = {}
    group['total_dosage'] = group.groupby('SVC_DT')['dosage'].transform('sum')
    for period in periods:
        # Calculate the cutoff date from the first 'episode.start' plus the period in months
        cutoff_date = group['episode.start'].iloc[0] + pd.DateOffset(months=period)
        # Filter dosages up to the cutoff date and calculate the max dosage
        max_dosage = group[group['SVC_DT'] <= cutoff_date]['total_dosage'].max()
        results[f'max_dosage_{period}M'] = max_dosage
    results['uuid'] = uuid  # Add the uuid to the results
    return pd.Series(results)

# Apply the function to each group and combine the results into a DataFrame
max_dosage_results = groupedby_PI_ES.apply(calculate_max_dosage, periods=required_months)

# Ensure the results DataFrame has one row per group, reset the index if needed
max_dosage_results.reset_index(drop=True, inplace=True)

# Merge the calculated max dosages back 
# This assumes 'uuid' is a column in episode_RX that can be used to merge
episode_RX_var = pd.merge(episode_RX_keep, max_dosage_results, on='uuid', how='left')


In [19]:
# Check number of rows
print(len(episode_RX_var))
print(len(max_dosage_results))

15606863
1213812


In [20]:
# Check number of unique episodes
len(set(episode_RX_var['uuid']))

1213812

In [21]:
# Save the data
episode_RX_var.to_csv('file path',index=False)